I took these data file from [kaggle](https://www.kaggle.com/dgawlik/nyse/downloads/nyse.zip/3) to play around with fundamental and technical analysis. There are four (4) data sets to explore:

- `prices.csv`: raw, as-is daily prices. Most of the data spans from 2010 to the end of 2016. There have been approx. 140 stock splits in that time but this data set does not reflect that
- `prices-split-adjusted.csv`: same as `prices.csv` but adjustments for splits have been added
- `securities.csv`: general description of each company with division on sectors
- `fundamentals.csv`: metrics extracted from annual SEC 10K fillings (2012-2016); should be enough to derive most of the popular fundamental indicators

Some things to try out with the data:

- One day ahead prediction: Rolling Linear Regression, ARIMA, Neural Networks, LSTM
- Momentum/Mean-Reversion Strategies
- Security clustering, portfolio construction/hedging

Which company has the biggest chance of going bankrupt?
Which one is undervalued (how prices behaved afterwards) and what is the ROI?

In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [33]:
path = "/Users/scottmaccarone/Desktop/Coding/my_fun_projects/nyse"

In [34]:
fundamentals = pd.read_csv(os.path.join(path, "fundamentals.csv"))

In [35]:
prices = pd.read_csv(os.path.join(path, "prices.csv"))

In [36]:
pr_spl_adj = pd.read_csv(os.path.join(path, "prices-split-adjusted.csv"))

In [37]:
securities = pd.read_csv(os.path.join(path, "securities.csv"))

In [38]:
fundamentals.shape

(1781, 79)

In [39]:
prices.shape

(851264, 7)

In [40]:
pr_spl_adj.shape

(851264, 7)

In [41]:
securities.shape

(505, 8)

In [42]:
# In case I end up joining all four dataframes, there would be a max of 101 columns
pd.options.display.max_columns = 101

# To ensure columns fit well, I'll adjust the max column length
pd.options.display.max_colwidth = 100

In [43]:
fundamentals.head(10)

,Unnamed: 0,Ticker Symbol,Period Ending,Accounts Payable,Accounts Receivable,Add'l income/expense items,After Tax ROE,Capital Expenditures,Capital Surplus,Cash Ratio,Cash and Cash Equivalents,Changes in Inventories,Common Stocks,Cost of Revenue,Current Ratio,Deferred Asset Charges,Deferred Liability Charges,Depreciation,Earnings Before Interest and Tax,Earnings Before Tax,Effect of Exchange Rate,Equity Earnings/Loss Unconsolidated Subsidiary,Fixed Assets,Goodwill,Gross Margin,Gross Profit,Income Tax,Intangible Assets,Interest Expense,Inventory,Investments,Liabilities,Long-Term Debt,Long-Term Investments,Minority Interest,Misc. Stocks,Net Borrowings,Net Cash Flow,Net Cash Flow-Operating,Net Cash Flows-Financing,Net Cash Flows-Investing,Net Income,Net Income Adjustments,Net Income Applicable to Common Shareholders,Net Income-Cont. Operations,Net Receivables,Non-Recurring Items,Operating Income,Operating Margin,Other Assets,Other Current Assets,Other Current Liabilities,Other Equity,Other Financing Activities,Other Investing Activities,Other Liabilities,Other Operating Activities,Other Operating Items,Pre-Tax Margin,Pre-Tax ROE,Profit Margin,Quick Ratio,Research and Development,Retained Earnings,Sale and Purchase of Stock,"Sales, General and Admin.",Short-Term Debt / Current Portion of Long-Term Debt,Short-Term Investments,Total Assets,Total Current Assets,Total Current Liabilities,Total Equity,Total Liabilities,Total Liabilities & Equity,Total Revenue,Treasury Stock,For Year,Earnings Per Share,Estimated Shares Outstanding
0,0,AAL,2012-12-31,3.068000e+09,-2.220000e+08,-1.961000e+09,23.0,-1.888000e+09,4.695000e+09,53.0,1.330000e+09,0.0,1.270000e+08,1.049900e+10,78.0,0.000000e+00,2.230000e+08,1.001000e+09,-1.813000e+09,-2.445000e+09,0.0,0.0,1.340200e+10,0.000000e+00,58.0,1.435600e+10,-5.690000e+08,8.690000e+08,632000000.0,5.800000e+08,3.060000e+08,4.730000e+08,7.116000e+09,0.000000e+00,0.0,0.0,-1.020000e+09,1.970000e+08,1.285000e+09,4.830000e+08,-1.571000e+09,-1.876000e+09,2.050000e+09,-1.876000e+09,-4.084000e+09,1.124000e+09,3.860000e+08,1.480000e+08,1.0,2.167000e+09,6.260000e+08,4.524000e+09,-2.980000e+09,1.509000e+09,1.100000e+07,1.514700e+10,-1.410000e+08,8.450000e+08,10.0,31.0,8.0,72.0,0.000000e+00,-9.462000e+09,0.000000e+00,1.297700e+10,1.419000e+09,3.412000e+09,2.351000e+10,7.072000e+09,9.011000e+09,-7.987000e+09,2.489100e+10,1.690400e+10,2.485500e+10,-367000000.0,2012.0,-5.60,3.350000e+08
1,1,AAL,2013-12-31,4.975000e+09,-9.300000e+07,-2.723000e+09,67.0,-3.114000e+09,1.059200e+10,75.0,2.175000e+09,0.0,5.000000e+06,1.101900e+10,104.0,0.000000e+00,9.350000e+08,1.020000e+09,-1.324000e+09,-2.180000e+09,0.0,0.0,1.925900e+10,4.086000e+09,59.0,1.572400e+10,-3.460000e+08,2.311000e+09,856000000.0,1.012000e+09,-1.181000e+09,-2.350000e+08,1.535300e+10,0.000000e+00,0.0,0.0,2.208000e+09,6.600000e+08,6.750000e+08,3.799000e+09,-3.814000e+09,-1.834000e+09,1.873000e+09,-1.834000e+09,-4.489000e+09,1.560000e+09,5.590000e+08,1.399000e+09,5.0,2.299000e+09,1.465000e+09,7.385000e+09,-2.032000e+09,1.711000e+09,4.810000e+08,1.491500e+10,-5.600000e+07,8.530000e+08,8.0,80.0,7.0,96.0,0.000000e+00,-1.129600e+10,0.000000e+00,1.291300e+10,1.446000e+09,8.111000e+09,4.227800e+10,1.432300e+10,1.380600e+10,-2.731000e+09,4.500900e+10,4.227800e+10,2.674300e+10,0.0,2013.0,-11.25,1.630222e+08
2,2,AAL,2014-12-31,4.668000e+09,-1.600000e+08,-1.500000e+08,143.0,-5.311000e+09,1.513500e+10,60.0,1.768000e+09,0.0,7.000000e+06,1.562000e+10,88.0,0.000000e+00,8.290000e+08,1.342000e+09,4.099000e+09,3.212000e+09,0.0,0.0,2.308400e+10,4.091000e+09,63.0,2.703000e+10,3.300000e+08,2.240000e+09,887000000.0,1.004000e+09,1.799000e+09,-1.026000e+09,1.604300e+10,0.000000e+00,0.0,0.0,1.700000e+08,-1.460000e+08,3.080000e+09,-3.150000e+08,-2.911000e+09,2.882000e+09,5.420000e+08,2.882000e+09,2.882000e+09,1.771000e+09,8.000000e+08,4.249000e+09,10.0,2.060000e+09,8.980000e+08,7.059000e+09,-4.559000e+09,8.170000e+08,6.010000e+08,1.092800e+10,-5.000000e+08,1.295000e+09,8.0,159.0,7.0,80.0,0.0000

In [44]:
prices.head(10)

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0
5,2016-01-12 00:00:00,WLTW,115.510002,115.550003,114.500000,116.059998,1098000.0
6,2016-01-13 00:00:00,WLTW,116.459999,112.849998,112.589996,117.070000,949600.0
7,2016-01-14 00:00:00,WLTW,113.510002,114.379997,110.050003,115.029999,785300.0
8,2016-01-15 00:00:00,WLTW,113.330002,112.529999,111.919998,114.879997,1093700.0
9,2016-01-19 00:00:00,WLTW,113.660004,110.379997,109.870003,115.870003,1523500.0


In [45]:
pr_spl_adj.head(10)

,date,symbol,open,close,low,high,volume
0,2016-01-05,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0
5,2016-01-12,WLTW,115.510002,115.550003,114.500000,116.059998,1098000.0
6,2016-01-13,WLTW,116.459999,112.849998,112.589996,117.070000,949600.0
7,2016-01-14,WLTW,113.510002,114.379997,110.050003,115.029999,785300.0
8,2016-01-15,WLTW,113.330002,112.529999,111.919998,114.879997,1093700.0
9,2016-01-19,WLTW,113.660004,110.379997,109.870003,115.870003,1523500.0


In [46]:
securities.head(10)

,Ticker symbol,Security,SEC filings,GICS Sector,GICS Sub Industry,Address of Headquarters,Date first added,CIK
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",NaN,66740
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800
2,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152
3,ACN,Accenture plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373
4,ATVI,Activision Blizzard,reports,Information Technology,Home Entertainment Software,"Santa Monica, California",2015-08-31,718877
5,AYI,Acuity Brands Inc,reports,Industrials,Electrical Components & Equipment,"Atlanta, Georgia",2016-05-03,1144215
6,ADBE,Adobe Systems Inc,reports,Information Technology,Application Software,"San Jose, California",1997-05-05,796343
7,AAP,Advance Auto Parts,reports,Consumer Discretionary,Automotive Retail,"Roanoke, Virginia",2015-07-09,1158449
8,AES,AES Corp,reports,Utilities,Independent Power Producers & Energy Traders,"Arlington, Virginia",NaN,874761
9,AET,Aetna Inc,reports,Health Care,Managed Health Care,"Hartford, Connecticut",1976-06-30,1122304


Some notes regarding cleaning and first impressions:

- The `fundamentals` data table contains a lot of scientific notation. I want to check the data types and see if I can convert back to just `float`
- `Date` information is in `YYYY-MM-DD` format (or maybe `YYYY-mm-dd` - I need to refresh my memory on this). The `prices` df includes time information as well, but it is unclear if the time is always `00:00:00`
-  The `CIK` column is the **Central Index Key** which is a **unique number** given to an organization by the SEC to identify that organizations filings in several online databases
- Is the `SEC filings` column useful?
- Remove the `Unnamed:0` column
- ALL tables have the ticker symbol, so joining these tables is possible

In [47]:
securities['SEC filings'].value_counts()

reports    505
Name: SEC filings, dtype: int64

In [48]:
securities.isnull().sum()

Ticker symbol                0
Security                     0
SEC filings                  0
GICS Sector                  0
GICS Sub Industry            0
Address of Headquarters      0
Date first added           198
CIK                          0
dtype: int64

The `securities` table has very useful information. The only column that seems to be rather useless is the `SEC filings` column where ALL entries/rows are `reports`. The `Date first added` column has a lot of missing/null data; however, I don't want to remove this column yet because this column can tell us how old a company is. My thought is that VERY OLD companies have a `Date first added` as `NaN`.

In [49]:
securities.drop(labels='SEC filings', axis=1, inplace=True)
securities.head()

,Ticker symbol,Security,GICS Sector,GICS Sub Industry,Address of Headquarters,Date first added,CIK
0,MMM,3M Company,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",NaN,66740
1,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800
2,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152
3,ACN,Accenture plc,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373
4,ATVI,Activision Blizzard,Information Technology,Home Entertainment Software,"Santa Monica, California",2015-08-31,718877


In [53]:
prices['date'].value_counts(dropna=False).sort_index()

2010-01-04             466
2010-01-04 00:00:00      1
2010-01-05             467
2010-01-05 00:00:00      1
2010-01-06             467
2010-01-06 00:00:00      1
2010-01-07             467
2010-01-07 00:00:00      1
2010-01-08             467
2010-01-08 00:00:00      1
2010-01-11             467
2010-01-11 00:00:00      1
2010-01-12             467
2010-01-12 00:00:00      1
2010-01-13             467
2010-01-13 00:00:00      1
2010-01-14             467
2010-01-14 00:00:00      1
2010-01-15             467
2010-01-15 00:00:00      1
2010-01-19             467
2010-01-19 00:00:00      1
2010-01-20             467
2010-01-20 00:00:00      1
2010-01-21             467
2010-01-21 00:00:00      1
2010-01-22             467
2010-01-22 00:00:00      1
2010-01-25             467
2010-01-25 00:00:00      1
                      ... 
2016-12-09             498
2016-12-09 00:00:00      3
2016-12-12             498
2016-12-12 00:00:00      3
2016-12-13             498
2016-12-13 00:00:00      3
2

The `date` column for `prices` is odd, and seems to be inconsistent. Why the difference between `YYYY-MM-DD` and `YYYY-MM-DD HH:MM:SS`? I am thinking the `HH:MM:SS` components can be dropped.

In [54]:
prices['date'][0]

'2016-01-05 00:00:00'

In [55]:
type(prices['date'][0])

str

In [56]:
len(prices['date'][0])

19

The length of the strings include the white-space between `YYYY-MM-DD` and `HH:MM:SS`. I can use string selection to select and return on the first 10 characters. Can this be done with vectorized operations? Since I am working in pandas, I assume there is a way to do it. Once that is complete, I can convert the date column into a `datetime` object and make some time-series graphs.

I do wonder if there is a particular ticker symbol associated with the `HH:MM:SS` entries. One thought that comes to mind is to define a function that determines if a particular row in the pandas dataframe has `len(prices['date'][i])` (the entries are strings) greater than `10`. If so, print only the unique value. Then I can use the `df.apply()` method to vectorize this function.

In [70]:
def date_len(val):
    if len(prices['date'][val]) > 10:
        print(prices.iloc[val, :])

I can't seem to get the `prices.apply(date_len)` to work properly (I keep getting errors). My function could be defined incorrectly, or maybe I'm trying to apply the function incorrectly.

In [77]:
#unique_rows = []
#i=0
#while i <= prices.shape[0]:
#    if len(prices['date'][i]) > 10:
#        if prices['symbol'][i] not in unique_rows:
#            unique_rows.append(prices.iloc[i, :])
#    i += 1
#print(unique_rows)

In [85]:
unique_sym = prices[prices['date'].str.len() > 10]['symbol'].unique()
unique_sym

array(['WLTW', 'AIV', 'FTV'], dtype=object)

That was a lot easier than I was making it out to be! Let me check what these companies are:

In [88]:
type(unique_sym)

numpy.ndarray

I want to select rows from `securities` and `fundamentals` to better understand these three companies